In [1]:
import os
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

In [2]:
# Set working directory
os.chdir(r"D:\Benson\aUpWork\Ben Ruff\Implementation\Returns Viz App")

# Import the CSV file
dia_data = pd.read_csv(r"Historical Data\DIA.csv")
spy_data = pd.read_csv(r"Historical Data\SPY.csv")
qqq_data = pd.read_csv(r"Historical Data\QQQ.csv")
vti_data = pd.read_csv(r"Historical Data\VTI.csv")

In [3]:
# Convert Date columns to datetime and clean up dataframes
dia_data['Date'] = pd.to_datetime(dia_data['Date'])
spy_data['Date'] = pd.to_datetime(spy_data['Date'])
qqq_data['Date'] = pd.to_datetime(qqq_data['Date'])
vti_data['Date'] = pd.to_datetime(vti_data['Date'])

# Keep only Date and Adj Close columns, rename Adj Close to ETF name
dia_data = dia_data[['Date', 'Adj Close']].rename(columns={'Adj Close': 'DIA'})
spy_data = spy_data[['Date', 'Adj Close']].rename(columns={'Adj Close': 'SPY'})
qqq_data = qqq_data[['Date', 'Adj Close']].rename(columns={'Adj Close': 'QQQ'})
vti_data = vti_data[['Date', 'Adj Close']].rename(columns={'Adj Close': 'VTI'})

In [4]:
# Convert prices to returns
dia_data['DIA'] = dia_data['DIA'].pct_change()
spy_data['SPY'] = spy_data['SPY'].pct_change()
qqq_data['QQQ'] = qqq_data['QQQ'].pct_change()
vti_data['VTI'] = vti_data['VTI'].pct_change()

# Drop the first row with NaN values from pct_change()
dia_data = dia_data.dropna()
spy_data = spy_data.dropna()
qqq_data = qqq_data.dropna()
vti_data = vti_data.dropna()

In [5]:
# Create a table of annual returns for each year for all 4 ETFs

# Combine all dataframes
combined_data = dia_data.merge(spy_data, on='Date', how='outer') \
                        .merge(qqq_data, on='Date', how='outer') \
                        .merge(vti_data, on='Date', how='outer')

# Set Date as index and sort chronologically
combined_data.set_index('Date', inplace=True)
combined_data.sort_index(inplace=True)  # CRITICAL: Sort by date for proper compounding

# Extract year from the date index
combined_data['Year'] = combined_data.index.year

# Quick data validation
print(f"Date range: {combined_data.index.min().date()} to {combined_data.index.max().date()}")
print(f"Total trading days: {len(combined_data)}")
print(f"Missing data by ETF:")
for etf in ['DIA', 'SPY', 'QQQ', 'VTI']:
    missing_pct = (combined_data[etf].isna().sum() / len(combined_data)) * 100
    print(f"  {etf}: {missing_pct:.1f}% missing")

# Group by year and calculate annual returns correctly
# Compound all daily returns for each year: (1+r1) × (1+r2) × ... × (1+rn) - 1
# Note: .prod() automatically handles NaN values by excluding them
annual_returns = combined_data.groupby('Year')[['DIA', 'SPY', 'QQQ', 'VTI']].apply(
    lambda x: (1 + x).prod() - 1
)

# Convert to percentage and round to 2 decimal places
annual_returns = (annual_returns * 100).round(2)

# Display the table
print("Annual Returns by Year (%)")
print("=" * 40)

display(annual_returns)


Date range: 1993-02-01 to 2025-07-01
Total trading days: 8160
Missing data by ETF:
  DIA: 15.4% missing
  SPY: 0.0% missing
  QQQ: 18.9% missing
  VTI: 25.9% missing
Annual Returns by Year (%)


,DIA,SPY,QQQ,VTI
Year,,,,
1993,0.00,8.71,0.00,0.00
1994,0.00,0.40,0.00,0.00
1995,0.00,38.05,0.00,0.00
1996,0.00,22.50,0.00,0.00
1997,0.00,33.48,0.00,0.00
1998,17.76,28.69,0.00,0.00
1999,27.71,20.39,78.95,0.00
2000,-6.05,-9.74,-36.11,0.00
2001,-4.97,-11.76,-33.34,-4.40


# Making the Returns Matrix

In [6]:
def create_cagr_matrix(annual_returns_series, start_year=None, end_year=None):
    """
    Creates a matrix of Compound Annual Growth Rates (CAGR) between different year ranges.
    
    Parameters:
    annual_returns_series: Series with annual returns as percentages, indexed by year
    start_year: First year to include (default: earliest year in data)
    end_year: Last year to include (default: latest year in data)
    
    Returns:
    DataFrame where columns are starting years, rows are ending years,
    and values are CAGR percentages
    """
    # Convert percentages back to decimals for calculations
    returns_decimal = annual_returns_series / 100
    
    # Get year range
    if start_year is None:
        start_year = returns_decimal.index.min()
    if end_year is None:
        end_year = returns_decimal.index.max()
    
    # Filter data to specified year range
    returns_filtered = returns_decimal.loc[start_year:end_year]
    years = returns_filtered.index.tolist()
    
    # Initialize matrix
    matrix = pd.DataFrame(index=years, columns=years, dtype=float)
    
    # Fill the matrix
    for start_yr in years:
        for end_yr in years:
            if end_yr >= start_yr:  # Only calculate for valid date ranges
                # Get returns for the period
                period_returns = returns_filtered.loc[start_yr:end_yr]
                
                # Calculate number of years
                num_years = len(period_returns)
                
                if num_years == 1:
                    # Single year return
                    cagr = period_returns.loc[start_yr] * 100
                else:
                    # Multi-year CAGR: ((1+r1) * (1+r2) * ... * (1+rn))^(1/n) - 1
                    cumulative_return = (1 + period_returns).prod()
                    cagr = (cumulative_return ** (1/num_years) - 1) * 100
                
                matrix.loc[end_yr, start_yr] = cagr
    
    return matrix


In [7]:

# Create CAGR matrices for each ETF
print("\n" + "="*60)
print("COMPOUND ANNUAL GROWTH RATE (CAGR) MATRICES")
print("="*60)

etfs = ['SPY', 'QQQ', 'DIA', 'VTI']

# You can adjust these years to focus on specific periods
# For example, start from 2013 to match your screenshot
matrix_start_year = 2013
matrix_end_year = 2024

for etf in etfs:
    print(f"\n{etf} - CAGR Matrix ({matrix_start_year}-{matrix_end_year})")
    print("-" * 50)
    
    # Generate CAGR matrix for this ETF
    cagr_matrix = create_cagr_matrix(
        annual_returns[etf], 
        start_year=matrix_start_year, 
        end_year=matrix_end_year
    )
    
    # Round to 2 decimal places and display
    cagr_matrix_rounded = cagr_matrix.round(2)
    
    # Replace NaN with empty strings for cleaner display
    display_matrix = cagr_matrix_rounded.fillna('')
    
    # Display the matrix
    display(display_matrix)




#        DIA     SPY    QQQ     VTI
#2024	14.82	24.89	25.58	23.81
#2025	5.52	5.90	7.02	5.50





COMPOUND ANNUAL GROWTH RATE (CAGR) MATRICES

SPY - CAGR Matrix (2013-2024)
--------------------------------------------------


,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
2013,32.31,,,,,,,,,,,
2014,22.52,13.46,,,,,,,,,,
2015,14.97,7.17,1.23,,,,,,,,,
2016,14.22,8.76,6.48,12.0,,,,,,,,
2017,15.68,11.86,11.33,16.75,21.71,,,,,,,
2018,12.03,8.36,7.12,9.16,7.77,-4.57,,,,,,
2019,14.59,11.87,11.56,14.3,15.08,11.9,31.22,,,,,
2020,15.05,12.77,12.66,15.1,15.88,14.01,24.61,18.33,,,,
2021,16.49,14.66,14.83,17.26,18.35,17.52,25.97,23.42,28.73,,,
2022,12.45,10.44,10.06,11.39,11.29,9.31,13.09,7.62,2.63,-18.18,,



QQQ - CAGR Matrix (2013-2024)
--------------------------------------------------


,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
2013,36.63,,,,,,,,,,,
2014,27.61,19.18,,,,,,,,,,
2015,21.24,14.21,9.44,,,,,,,,,
2016,17.54,11.79,8.26,7.1,,,,,,,,
2017,20.42,16.67,15.85,19.2,32.66,,,,,,,
2018,16.72,13.1,11.63,12.37,15.1,-0.13,,,,,,
2019,19.67,17.05,16.63,18.5,22.56,17.8,38.96,,,,,
2020,22.93,21.09,21.41,23.95,28.57,27.23,43.61,48.41,,,,
2021,23.42,21.86,22.25,24.53,28.34,27.28,38.0,37.52,27.42,,,
2022,16.18,14.1,13.49,14.08,15.28,12.09,15.37,8.43,-7.31,-32.58,,



DIA - CAGR Matrix (2013-2024)
--------------------------------------------------


,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
2013,29.64,,,,,,,,,,,
2014,19.26,9.72,,,,,,,,,,
2015,12.50,4.79,0.09,,,,,,,,,
2016,13.45,8.52,7.92,16.37,,,,,,,,
2017,16.24,13.11,14.26,22.08,28.08,,,,,,,
2018,12.64,9.52,9.47,12.79,11.04,-3.74,,,,,,
2019,14.33,11.96,12.42,15.73,15.52,9.71,25.03,,,,,
2020,13.73,11.62,11.94,14.48,14.01,9.67,17.06,9.59,,,,
2021,14.50,12.73,13.17,15.51,15.34,12.36,18.3,15.07,20.83,,,
2022,12.14,10.35,10.42,11.98,11.27,8.18,11.39,7.18,5.99,-7.02,,



VTI - CAGR Matrix (2013-2024)
--------------------------------------------------


,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
2013,33.45,,,,,,,,,,,
2014,22.56,12.55,,,,,,,,,,
2015,14.66,6.28,0.36,,,,,,,,,
2016,14.20,8.42,6.41,12.82,,,,,,,,
2017,15.57,11.48,11.13,16.94,21.21,,,,,,,
2018,11.81,7.92,6.79,9.03,7.18,-5.23,,,,,,
2019,14.33,11.42,11.19,14.08,14.5,11.28,30.67,,,,,
2020,15.15,12.75,12.78,15.44,16.11,14.46,25.78,21.08,,,,
2021,16.27,14.29,14.54,17.09,17.96,17.16,25.75,23.36,25.68,,,
2022,12.07,9.92,9.6,10.98,10.68,8.69,12.47,6.99,0.57,-19.52,,


In [8]:
# Create a more comprehensive analysis with different time periods
print("\nCOMPREHENSIVE CAGR ANALYSIS")
print("="*50)

# Create matrices for different time periods
time_periods = {
    "Full Period (1993-2025)": (1993, 2025),
    "Modern Era (2000-2025)": (2000, 2025), 
    "Post-Crisis (2010-2025)": (2010, 2025),
    "Recent Decade (2013-2024)": (2013, 2024)
}

# Store all matrices for potential export
all_matrices = {}

for period_name, (start_yr, end_yr) in time_periods.items():
    print(f"\n{period_name}")
    print("-" * len(period_name))
    
    period_matrices = {}
    
    for etf in etfs:
        # Skip if ETF doesn't have data for this period
        if annual_returns[etf].loc[start_yr:end_yr].isna().all():
            continue
            
        cagr_matrix = create_cagr_matrix(
            annual_returns[etf], 
            start_year=start_yr, 
            end_year=end_yr
        )
        period_matrices[etf] = cagr_matrix.round(2)
    
    all_matrices[period_name] = period_matrices
    
    # Display a summary table for this period
    print(f"Available ETFs: {list(period_matrices.keys())}")

# Save matrices to Excel file for easier viewing and analysis
try:
    import openpyxl
    with pd.ExcelWriter('CAGR_Matrices_Complete.xlsx', engine='openpyxl') as writer:
        # First, save the raw daily returns data
        combined_data[['DIA', 'SPY', 'QQQ', 'VTI']].to_excel(writer, sheet_name='Daily_Returns')
        
        # Second, save the annual returns
        annual_returns.to_excel(writer, sheet_name='Annual_Returns')
        
        # Save each time period's matrices
        for period_name, period_matrices in all_matrices.items():
            # Create a clean sheet name
            sheet_name = period_name.replace("(", "").replace(")", "").replace(" ", "_")[:31]
            
            # Combine all ETFs for this period into one sheet
            start_row = 0
            for etf, matrix in period_matrices.items():
                # Write ETF name
                temp_df = pd.DataFrame([f"{etf} - {period_name}"])
                temp_df.to_excel(writer, sheet_name=sheet_name, startrow=start_row, 
                                index=False, header=False)
                start_row += 2
                
                # Write the matrix
                matrix.fillna('').to_excel(writer, sheet_name=sheet_name, startrow=start_row)
                start_row += len(matrix) + 3
    
    print(f"\nAll CAGR matrices saved to 'CAGR_Matrices_Complete.xlsx'")
    print("Each sheet contains matrices for different time periods.")
    
except ImportError:
    print("\nNote: openpyxl not available. Install with: pip install openpyxl")
except Exception as e:
    print(f"Note: Could not save to Excel file. Error: {e}")

# Display a focused matrix for SPY (as an example, matching your screenshot style)
print(f"\n\nSPY CAGR MATRIX - Recent Decade (2013-2024)")
print("=" * 50)
print("Rows = End Year, Columns = Start Year")
print("Values = Compound Annual Growth Rate (%)")

spy_matrix = create_cagr_matrix(annual_returns['SPY'], start_year=2013, end_year=2024)
spy_display = spy_matrix.round(2).fillna('')

# Reverse the row order to match typical matrix display (latest year at top)
spy_display_reversed = spy_display.iloc[::-1]
display(spy_display_reversed)



COMPREHENSIVE CAGR ANALYSIS

Full Period (1993-2025)
-----------------------
Available ETFs: ['SPY', 'QQQ', 'DIA', 'VTI']

Modern Era (2000-2025)
----------------------
Available ETFs: ['SPY', 'QQQ', 'DIA', 'VTI']

Post-Crisis (2010-2025)
-----------------------
Available ETFs: ['SPY', 'QQQ', 'DIA', 'VTI']

Recent Decade (2013-2024)
-------------------------
Available ETFs: ['SPY', 'QQQ', 'DIA', 'VTI']

All CAGR matrices saved to 'CAGR_Matrices_Complete.xlsx'
Each sheet contains matrices for different time periods.


SPY CAGR MATRIX - Recent Decade (2013-2024)
Rows = End Year, Columns = Start Year
Values = Compound Annual Growth Rate (%)


,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
2024,14.53,13.04,13.0,14.39,14.69,13.72,17.09,14.45,13.5,8.84,25.53,24.89
2023,13.63,11.92,11.75,13.14,13.3,11.96,15.59,11.98,9.95,1.61,26.18,
2022,12.45,10.44,10.06,11.39,11.29,9.31,13.09,7.62,2.63,-18.18,,
2021,16.49,14.66,14.83,17.26,18.35,17.52,25.97,23.42,28.73,,,
2020,15.05,12.77,12.66,15.1,15.88,14.01,24.61,18.33,,,,
2019,14.59,11.87,11.56,14.3,15.08,11.9,31.22,,,,,
2018,12.03,8.36,7.12,9.16,7.77,-4.57,,,,,,
2017,15.68,11.86,11.33,16.75,21.71,,,,,,,
2016,14.22,8.76,6.48,12.0,,,,,,,,
2015,14.97,7.17,1.23,,,,,,,,,


In [9]:
# Example: Create a custom matrix for comparison (e.g., all ETFs side by side for 2013-2024)
print("COMPARISON MATRIX - All ETFs (2013-2024)")
print("="*50)

# Create a combined comparison matrix
comparison_year_start = 2013
comparison_year_end = 2024

fig_data = {}
for etf in etfs:
    matrix = create_cagr_matrix(annual_returns[etf], comparison_year_start, comparison_year_end)
    fig_data[etf] = matrix.round(2)

# You can also create specific slices for analysis
print("\nSample Analysis: 10-Year Rolling CAGR ending in 2024")
print("-" * 50)

ten_year_cagr = {}
for etf in etfs:
    if not annual_returns[etf].loc[2014:2024].isna().all():
        matrix = create_cagr_matrix(annual_returns[etf], 2014, 2024)
        ten_year_return = matrix.loc[2024, 2014]
        ten_year_cagr[etf] = ten_year_return

ten_year_df = pd.DataFrame.from_dict(ten_year_cagr, orient='index', columns=['10-Year CAGR (2014-2024)'])
ten_year_df = ten_year_df.round(2)
print("\n10-Year CAGR ending December 2024:")
display(ten_year_df)

# Usage examples and tips
print("\n" + "="*60)
print("USAGE EXAMPLES")
print("="*60)
print("""
1. To create a matrix for a specific ETF and time period:
   matrix = create_cagr_matrix(annual_returns['SPY'], start_year=2010, end_year=2023)

2. To get a specific CAGR (e.g., SPY from 2015 to 2020):
   matrix = create_cagr_matrix(annual_returns['SPY'], 2015, 2020)
   cagr_2015_to_2020 = matrix.loc[2020, 2015]

3. Matrix interpretation:
   - Columns = Starting year of investment
   - Rows = Ending year of investment  
   - Values = Compound Annual Growth Rate (%)
   
4. The diagonal shows single-year returns
   
5. Upper triangle is empty (can't end before you start)
   
6. Lower triangle shows multi-year CAGRs
""")


COMPARISON MATRIX - All ETFs (2013-2024)

Sample Analysis: 10-Year Rolling CAGR ending in 2024
--------------------------------------------------

10-Year CAGR ending December 2024:


,10-Year CAGR (2014-2024)
SPY,13.04
QQQ,18.34
DIA,11.25
VTI,12.51



USAGE EXAMPLES

1. To create a matrix for a specific ETF and time period:
   matrix = create_cagr_matrix(annual_returns['SPY'], start_year=2010, end_year=2023)

2. To get a specific CAGR (e.g., SPY from 2015 to 2020):
   matrix = create_cagr_matrix(annual_returns['SPY'], 2015, 2020)
   cagr_2015_to_2020 = matrix.loc[2020, 2015]

3. Matrix interpretation:
   - Columns = Starting year of investment
   - Rows = Ending year of investment  
   - Values = Compound Annual Growth Rate (%)
   
4. The diagonal shows single-year returns
   
5. Upper triangle is empty (can't end before you start)
   
6. Lower triangle shows multi-year CAGRs

